In [120]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../data_prep')
from tmdbv3api import TMDb
from tmdbv3api import Movie, TV
from sqlite_utils import get_from_table, delete_records, insert_record_into_table, df_to_table
from dotenv import load_dotenv

load_dotenv()

True

In [121]:
tmdb_api_key = os.getenv('TMDB_API_KEY')
tmdb = TMDb()
tmdb.api_key = tmdb_api_key

In [122]:
attrs = {
    'belongs_to_collection': None,
    'budget': None,
    'original_language': 'languages',
    'popularity': 'popularity',
    'release_date': 'first_air_date',
    'revenue': None,
    'runtime': ['episode_run_time', 'number_of_episodes'],
    'status': 'status',
    'vote_average': 'vote_average',
    'vote_count': 'vote_count',
    'keywords': 'keywords',
    'casts': 'credits'
}

In [143]:
def create_movie_metadata_dict(film_id):
    film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
    tmdb_id = get_from_table('TMDB_ID', film_id, 'TMDB_ID')
    content_type = get_from_table('CONTENT_TYPE', film_id, 'CONTENT_TYPE')
    print('Film ID is "{}", title is "{}", and TMDB id is "{}", while content type is "{}"'.format(film_id, film_title, tmdb_id, content_type))
    movie_metadata_dict = {'FILM_ID': film_id}
    if content_type == 'movie':
        movie = Movie()
        try:
            details = movie.details(tmdb_id)
            for k in attrs:
                movie_metadata_dict[k] = details.get(k, None)
        except:
            None
    elif content_type == 'tv':
        tv = TV()
        details = tv.details(tmdb_id, 'credits,keywords')
        for k in attrs:
            if type(attrs[k]) == list:
                movie_metadata_dict[k] = [details.get(x, None) for x in attrs[k]]
            else:
                movie_metadata_dict[k] = details.get(attrs[k], None)
        movie_metadata_dict['original_language'] = movie_metadata_dict['original_language'][0]
        movie_metadata_dict['runtime'] = movie_metadata_dict['runtime'][0][0] * movie_metadata_dict['runtime'][1]
        movie_metadata_dict['status'] = movie_metadata_dict['status'].replace('Returning Series', 'Released').replace('Ended', 'Released')
        if len(movie_metadata_dict.get('keywords', {'keywords': []}).get('keywords')) == 0:
               movie_metadata_dict['keywords']['keywords'] = [{'id': -1, 'name': 'none'}]
    else:
        details = {}
    return {'film_title': film_title, 'tmdb_id': tmdb_id, 'content_type':content_type, 'movie_metadata_dict': movie_metadata_dict, 'details': details}

In [148]:
f_0lrky = create_movie_metadata_dict('f_0lrky')
f_0lrky

Film ID is "f_0lrky", title is "The Kissing Booth 2", and TMDB id is "583083", while content type is "movie"
keywords {'keywords': []}


{'film_title': 'The Kissing Booth 2',
 'tmdb_id': '583083',
 'content_type': 'movie',
 'movie_metadata_dict': {'FILM_ID': 'f_0lrky',
  'belongs_to_collection': {'id': 726619, 'name': 'The Kissing Booth Collection', 'poster_path': '/wY0OtbE3WvZFnGNL37Jw28RvUSx.jpg', 'backdrop_path': '/6o99Ld3NBMBGbLXWoUWWm8Pq9LA.jpg'},
  'budget': 0,
  'original_language': 'en',
  'popularity': 28.985,
  'release_date': '2020-07-24',
  'revenue': 0,
  'runtime': 132,
  'status': 'Released',
  'vote_average': 7.727,
  'vote_count': 4724,
  'keywords': {'keywords': []},
  'casts': {'cast': [{'adult': False, 'gender': 1, 'id': 125025, 'known_for_department': 'Acting', 'name': 'Joey King', 'original_name': 'Joey King', 'popularity': 39.142, 'profile_path': '/kL0lWLJA6lbvmPM3YL0ISE6rVr6.jpg', 'cast_id': 1, 'character': 'Elle Evans', 'credit_id': '5f1abfe62afb25003414b13e', 'order': 0}, {'adult': False, 'gender': 2, 'id': 2034418, 'known_for_department': 'Acting', 'name': 'Jacob Elordi', 'original_name': 'Jac

In [149]:
f_0lrky['movie_metadata_dict'].get('keywords')

{'keywords': []}

In [145]:
keywords = f_0lrky[3].get('keywords', {'keywords': [{'id': -1, 'name': 'none'}]})
keywords


KeyError: 3

In [ ]:
if keywords:
    keywords = keywords.get('keywords')
    keyword_df = pd.DataFrame({
        'FILM_ID': [film_id]*len(keywords),
        'KEYWORD_ID':[x.get('id') for x in keywords],
        'KEYWORD':[x.get('name') for x in keywords],
        'CREATED_AT': [datetime.now()]*len(keywords)
                                })

In [136]:
tv = TV()
details = tv.details('5195', 'credits,keywords')

In [137]:
details

{'adult': False, 'backdrop_path': '/m8ksGiX010NWJt79iTPEuAzvHbt.jpg', 'created_by': [{'id': 5602, 'credit_id': '592ede95925141312000f4d0', 'name': 'David Lynch', 'gender': 2, 'profile_path': '/b6TnSpuqeOlbq7aHH9G4inmQ1v9.jpg'}], 'episode_run_time': [], 'first_air_date': '2002-01-01', 'genres': [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}], 'homepage': '', 'id': 5195, 'in_production': False, 'languages': ['en'], 'last_air_date': '2002-01-01', 'last_episode_to_air': {'id': 1432630, 'name': 'The Neighbor', 'overview': '', 'vote_average': 0.0, 'vote_count': 0, 'air_date': '2002-01-01', 'episode_number': 1, 'production_code': '', 'runtime': None, 'season_number': 1, 'show_id': 5195, 'still_path': None}, 'name': 'DumbLand', 'next_episode_to_air': None, 'networks': [], 'number_of_episodes': 8, 'number_of_seasons': 1, 'origin_country': ['US'], 'original_language': 'en', 'original_name': 'DumbLand', 'overview': 'A series of eight crudely animated shorts written, directed, and 

In [87]:
f_01UTC['details']['original_language']

'en'

In [88]:
for i in f_01UTC['details']:
    print(i)

adult
backdrop_path
belongs_to_collection
budget
genres
homepage
id
imdb_id
original_language
original_title
overview
popularity
poster_path
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
video
vote_average
vote_count
videos
trailers
images
casts
translations
keywords
release_dates


In [89]:
f_01U58['details']['languages']

['en']

In [118]:
f_01U58_2 = create_movie_metadata_dict('f_01U58')

Film ID is "f_01U58", title is "Salem's Lot", and TMDB id is "16118", while content type is "tv"


In [119]:
f_01U58_2['movie_metadata_dict']#['original_language']

{'FILM_ID': 'f_01U58',
 'belongs_to_collection': None,
 'budget': None,
 'original_language': 'en',
 'popularity': 20.037,
 'release_date': '1979-11-17',
 'revenue': None,
 'runtime': 184,
 'status': 'Ended',
 'vote_average': 6.9,
 'vote_count': 383,
 'keywords': {'results': [{'name': 'based on novel or book', 'id': 818}, {'name': 'vampire', 'id': 3133}, {'name': 'home', 'id': 1526}, {'name': 'return', 'id': 1571}, {'name': 'menace', 'id': 2250}, {'name': 'antiquary', 'id': 3095}, {'name': 'house', 'id': 6095}, {'name': 'miniseries', 'id': 11162}, {'name': 'writer', 'id': 13028}, {'name': 'new england', 'id': 41393}, {'name': 'hometown', 'id': 155808}, {'name': 'horror', 'id': 315058}]},
 'casts': {'cast': [{'adult': False, 'gender': 2, 'id': 15956, 'known_for_department': 'Acting', 'name': 'David Soul', 'original_name': 'David Soul', 'popularity': 8.281, 'profile_path': '/y7fW77RzeqgBYeoTswsMfwy8FwI.jpg', 'character': 'Ben Mears', 'credit_id': '52584c96760ee34661008d4c', 'order': 0}, 

In [90]:
f_01U58 = create_movie_metadata_dict('f_01U58')
for i in f_01U58['details']:
    print(i)

Film ID is "f_01U58", title is "Salem's Lot", and TMDB id is "16118", while content type is "tv"
adult
backdrop_path
created_by
episode_run_time
first_air_date
genres
homepage
id
in_production
languages
last_air_date
last_episode_to_air
name
next_episode_to_air
networks
number_of_episodes
number_of_seasons
origin_country
original_language
original_name
overview
popularity
poster_path
production_companies
production_countries
seasons
spoken_languages
status
tagline
type
vote_average
vote_count
credits
keywords


In [72]:
f_01U58['details']['episode_run_time'][0] * f_01U58['details']['number_of_episodes']

184

In [91]:
f_01U58['details']['keywords']

{'results': [{'name': 'based on novel or book', 'id': 818}, {'name': 'vampire', 'id': 3133}, {'name': 'home', 'id': 1526}, {'name': 'return', 'id': 1571}, {'name': 'menace', 'id': 2250}, {'name': 'antiquary', 'id': 3095}, {'name': 'house', 'id': 6095}, {'name': 'miniseries', 'id': 11162}, {'name': 'writer', 'id': 13028}, {'name': 'new england', 'id': 41393}, {'name': 'hometown', 'id': 155808}, {'name': 'horror', 'id': 315058}]}

In [48]:
film_id = 'f_01U58'
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
tmdb_id = get_from_table('TMDB_ID', film_id, 'TMDB_ID')
content_type = get_from_table('CONTENT_TYPE', film_id, 'CONTENT_TYPE')
print('Film ID is "{}", title is "{}", and TMDB id is "{}", while content type is "{}"'.format(film_id, film_title, tmdb_id, content_type))

Film ID is "f_01U58", title is "Salem's Lot", and TMDB id is "16118", while content type is "tv"


In [92]:
tv = TV()
details = tv.details('f_01U58', 'credits,keywords,budget')

TMDbException: The resource you requested could not be found.

In [161]:
from justwatch import JustWatch

just_watch = JustWatch(country='GB')
total_pages = just_watch.search_for_item(providers=['nfx'], content_types=['movie']).get('total_pages')

In [164]:
just_watch.search_for_item(providers=['nfx'], content_types=['movie']).get('items')

[{'jw_entity_id': 'tm860276',
  'id': 860276,
  'title': 'Nobody',
  'full_path': '/uk/movie/nobody',
  'full_paths': {'MOVIE_DETAIL_OVERVIEW': '/uk/movie/nobody'},
  'poster': '/poster/239506757/{profile}',
  'poster_blur_hash': 'TE9?:,WW0errayI;tRofni9aof-o',
  'object_type': 'movie',
  'scoring': [{'provider_type': 'jw:popular_1_day', 'value': 377.67},
   {'provider_type': 'imdb:votes', 'value': 280493},
   {'provider_type': 'jw:popular_180_day', 'value': 412.99},
   {'provider_type': 'jw:viral', 'value': 0.07},
   {'provider_type': 'imdb:score', 'value': 7.4},
   {'provider_type': 'tmdb:votes', 'value': 5813},
   {'provider_type': 'tmdb:id', 'value': 615457},
   {'provider_type': 'imdb:multiplied', 'value': 2075648.2000000002},
   {'provider_type': 'tmdb:score', 'value': 8.055},
   {'provider_type': 'tmdb:popularity', 'value': 83.242},
   {'provider_type': 'imdb:popularity', 'value': 35},
   {'provider_type': 'jw:popular_7_day', 'value': 478.3},
   {'provider_type': 'jw:popular_30_